# Implementación de campañas y filtros<a class="anchor" id="top"></a>

En este cuaderno, implementará e interactuará con campañas en Amazon Personalize.

1. [Introducción](#intro)
1. [Crear campañas](#create)
1. [Interactuar con campañas](#interact)
1. [Recomendaciones por lote](#batch)
1. [Conclusión](#wrapup)

## Introducción <a class="anchor" id="intro"></a>
[Regresar al principio](#top)

Hasta el momento, debería tener varias soluciones y al menos una versión de la solución para cada una de ellas. Una vez creada una versión de la solución, es posible obtener recomendaciones de la misma y hacerse una idea de su comportamiento general.

Este cuaderno comienza con la implementación de cada una de las versiones de la solución del cuaderno anterior en campañas individuales. Una vez que están activas, hay recursos para consultar las recomendaciones y funciones de ayuda para transformar el resultado en algo más legible para el ser humano. 

Al igual que con su cliente en Amazon Personalize, puede modificar las funciones de ayuda para que se ajusten a la estructura de sus archivos de entrada de datos y así mantener el funcionamiento de la representación adicional.

Para empezar, una vez más, necesitamos importar bibliotecas, cargar valores de cuadernos anteriores y cargar el SDK.

In [1]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random

import boto3
import botocore
from botocore.exceptions import ClientError
import pandas as pd

In [2]:
%store -r

In [3]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

## Crear campañas <a class="anchor" id="create"></a>
[Regresar al principio](#top)

Una campaña es una versión de la solución alojada; un punto de conexión que puede consultar para obtener recomendaciones. El precio se fija estimando la capacidad de rendimiento (solicitudes de personalización de los usuarios por segundo). Cuando implementa una campaña, se establece un valor mínimo de rendimiento por segundo (TPS). Este servicio, como muchos dentro de AWS, se escalará automáticamente en función de la demanda, pero si la latencia es crítica, es posible que desee aprovisionar por adelantado para una mayor demanda. Para esta POC y la demostración, todos los límites mínimos de rendimiento se establecen en 1. Para obtener más información, consulte la [página de precios](https://aws.amazon.com/personalize/pricing/).

Empecemos a implementar las campañas.

### Personalización del usuario

Implemente una campaña para su versión de la solución de personalización de usuarios. La implementación de una campaña puede tardar unos 10 minutos. Por lo general, utilizaríamos un bucle while para hacer un seguimiento hasta que se complete la tarea. Sin embargo, la tarea bloquearía la ejecución de otras celdas y el objetivo aquí es crear múltiples campañas. Así que estableceremos el bucle while para todas las campañas más adelante en el cuaderno. Allí, también encontrará instrucciones para ver el progreso en la consola de AWS.

In [4]:
userpersonalization_create_campaign_response = personalize.create_campaign(
    name = "personalize-poc-userpersonalization",
    solutionVersionArn = userpersonalization_solution_version_arn,
    minProvisionedTPS = 1
)

userpersonalization_campaign_arn = userpersonalization_create_campaign_response['campaignArn']
print(json.dumps(userpersonalization_create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:136455442858:campaign/personalize-poc-userpersonalization",
  "ResponseMetadata": {
    "RequestId": "ea0b63f5-9efa-4abd-bc88-dd5212e85668",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 17 Sep 2020 00:41:29 GMT",
      "x-amzn-requestid": "ea0b63f5-9efa-4abd-bc88-dd5212e85668",
      "content-length": "105",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### SIMS

Implemente una campaña para su versión de la solución SIMS. La implementación de una campaña puede tardar unos 10 minutos. Por lo general, utilizaríamos un bucle while para hacer un seguimiento hasta que se complete la tarea. Sin embargo, la tarea bloquearía la ejecución de otras celdas y el objetivo aquí es crear múltiples campañas. Así que estableceremos el bucle while para todas las campañas más adelante en el cuaderno. Allí, también encontrará instrucciones para ver el progreso en la consola de AWS.

In [5]:
sims_create_campaign_response = personalize.create_campaign(
    name = "personalize-poc-SIMS",
    solutionVersionArn = sims_solution_version_arn,
    minProvisionedTPS = 1
)

sims_campaign_arn = sims_create_campaign_response['campaignArn']
print(json.dumps(sims_create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:136455442858:campaign/personalize-poc-SIMS",
  "ResponseMetadata": {
    "RequestId": "8d8f06e3-2699-4bcd-bb60-9389aff0fcf4",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 17 Sep 2020 00:41:29 GMT",
      "x-amzn-requestid": "8d8f06e3-2699-4bcd-bb60-9389aff0fcf4",
      "content-length": "90",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Clasificación personalizada

Implemente una campaña para su versión de la solución de clasificación personalizada. La implementación de una campaña puede tardar unos 10 minutos. Por lo general, utilizaríamos un bucle while para hacer un seguimiento hasta que se complete la tarea. Sin embargo, la tarea bloquearía la ejecución de otras celdas y el objetivo aquí es crear múltiples campañas. Así que estableceremos el bucle while para todas las campañas más adelante en el cuaderno. Allí, también encontrará instrucciones para ver el progreso en la consola de AWS.

In [6]:
rerank_create_campaign_response = personalize.create_campaign(
    name = "personalize-poc-rerank",
    solutionVersionArn = rerank_solution_version_arn,
    minProvisionedTPS = 1
)

rerank_campaign_arn = rerank_create_campaign_response['campaignArn']
print(json.dumps(rerank_create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:136455442858:campaign/personalize-poc-rerank",
  "ResponseMetadata": {
    "RequestId": "c28bff59-c214-4f57-85d4-5e2643a03413",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 17 Sep 2020 00:41:29 GMT",
      "x-amzn-requestid": "c28bff59-c214-4f57-85d4-5e2643a03413",
      "content-length": "92",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Ver el estado de creación de la campaña

Según lo prometido, veremos cómo se pueden observar las actualizaciones de estado en la consola:

* En otra pestaña del navegador, ya debería tener la consola de AWS activa desde la apertura de esta instancia de cuaderno. 
* Cambie a esa pestaña y busque en la parte superior el servicio `Personalize`, luego diríjase a la página de ese servicio. 
* Haga clic en `View dataset groups` (Campañas).
* Haga clic en el nombre de su grupo de conjuntos de datos, probablemente contenga POC en el nombre.
* Haga clic en `Campaigns` (Campañas).
* Ahora verá una lista de todas las campañas que creó anteriormente, incluida una columna con el estado de las versiones de la solución. Una vez que esté `Active`, su campaña estará lista para ser consultada.

O simplemente ejecute la celda de abajo para seguir el estado de creación de la versión de la campaña.

In [7]:
in_progress_campaigns = [
    userpersonalization_campaign_arn,
    sims_campaign_arn,
    rerank_campaign_arn
]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for campaign_arn in in_progress_campaigns:
        version_response = personalize.describe_campaign(
            campaignArn = campaign_arn
        )
        status = version_response["campaign"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(campaign_arn))
            in_progress_campaigns.remove(campaign_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(campaign_arn))
            in_progress_campaigns.remove(campaign_arn)
    
    if len(in_progress_campaigns) <= 0:
        break
    else:
        print("At least one campaign build is still in progress")
        
    time.sleep(60)

At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
Build succeeded for arn:aws:personalize:us-east-1:136455442858:campaign/personalize-poc-SIMS
At least one campaign build is still in progress
Build succeeded for arn:aws:personalize:us-east-1:136455442858:campaign/personalize-poc-userpersonalization
At least one campaign build is still in progress
Build succeeded for arn:aws:personalize:us-east-1:136455442858:campaign/personalize-poc-rerank


## Crear filtros <a class="anchor" id="interact"></a>
[Regresar al principio](#top)

Ahora que todas las campañas están desplegadas y activas, podemos crear filtros. Se pueden crear filtros tanto para los elementos como para los eventos. Algunos casos de uso comunes de los filtros en los videos bajo demanda son los siguientes:

Filtros categóricos basados en los metadatos de los elementos: a menudo, los metadatos de los elementos contienen información sobre el título, como el género, las palabras clave, el año, la década, etc. El filtrado de estos datos puede proporcionar recomendaciones dentro de esos datos, como películas de acción.

Eventos: es posible que quiera filtrar ciertos eventos y proporcionar resultados basados en esos eventos, como mover un título de una recomendación de "sugerencias" a una recomendación de "ver de nuevo".

Veamos los metadatos de los elementos y las interacciones de los usuarios, para que podamos hacernos una idea del tipo de filtros que podemos crear.

In [8]:
# Create a dataframe for the items by reading in the correct source CSV
items_df = pd.read_csv(data_dir + '/item-meta.csv', sep=',', index_col=0)
#interactions_df = pd.read_csv(data_dir + '/interactions.csv', sep=',', index_col=0)

# Render some sample data
items_df.head(10)
#interactions_df.head(10)

,GENRE,YEAR
ITEM_ID,,
1,Adventure|Animation|Children|Comedy|Fantasy,1995
2,Adventure|Children|Fantasy,1995
3,Comedy|Romance,1995
4,Comedy|Drama|Romance,1995
5,Comedy,1995
6,Action|Crime|Thriller,1995
7,Comedy|Romance,1995
8,Adventure|Children,1995
9,Action,1995


Ahora bien, para determinar los géneros a filtrar, necesitamos una lista de todos los géneros. Primero obtendremos todos los valores únicos de la columna GENRE, luego dividiremos las cadenas en `|` si existen, todos se agregarán a una larga lista que se convertirá en un conjunto por eficiencia. Ese conjunto se convertirá en una lista para que pueda ser iterada y entonces podremos utilizar la API de creación de filtros.

In [9]:
unique_genre_field_values = items_df['GENRE'].unique()

genre_val_list = []

def process_for_bar_char(val, val_list):
    if '|' in val:
        values = val.split('|')
        for item in values:
            val_list.append(item)
    elif '(' in val:
        pass
    else:
        val_list.append(val)
    return val_list
    

for val in unique_genre_field_values:
    genre_val_list = process_for_bar_char(val, genre_val_list)

genres_to_filter = list(set(genre_val_list))

In [10]:
genres_to_filter

['Animation',
 'Thriller',
 'Sci-Fi',
 'IMAX',
 'Horror',
 'Comedy',
 'Adventure',
 'Musical',
 'Drama',
 'Romance',
 'Fantasy',
 'War',
 'Documentary',
 'Crime',
 'Western',
 'Film-Noir',
 'Mystery',
 'Action',
 'Children']

Con esto ya tenemos todos los géneros que existen en nuestro conjunto de datos. Un límite suave de Personalize en este momento es de 10 filtros totales, dado que tenemos un mayor número de géneros, seleccionaremos 7 al azar para dejar espacio para 2 filtros basados en la interacción más adelante y un filtro adicional para las recomendaciones basadas en el año.

In [11]:
genres_to_filter = random.sample(genres_to_filter, 7)
genres_to_filter

['Film-Noir', 'Fantasy', 'Western', 'Documentary', 'Comedy', 'Action', 'IMAX']

Ahora cree una lista para los filtros de género de metadatos y, luego, cree los filtros reales con las celdas que aparecen abajo. Tenga en cuenta que tardará unos minutos en completarse.

In [12]:
# Create a list for the filters:
meta_filter_arns = []

In [13]:
# Iterate through Genres
for genre in genres_to_filter:
    # Start by creating a filter
    try:
        createfilter_response = personalize.create_filter(
            name=genre,
            datasetGroupArn=dataset_group_arn,
            filterExpression='INCLUDE ItemID WHERE Items.GENRE IN ("'+ genre +'")'
        )
        # Add the ARN to the list
        meta_filter_arns.append(createfilter_response['filterArn'])
        print("Creating: " + createfilter_response['filterArn'])
    
    # If this fails, wait a bit
    except ClientError as error:
        # Here we only care about raising if it isnt the throttling issue
        if error.response['Error']['Code'] != 'LimitExceededException':
            print(error)
        else:    
            time.sleep(120)
            createfilter_response = personalize.create_filter(
                name=genre,
                datasetGroupArn=dataset_group_arn,
                filterExpression='INCLUDE ItemID WHERE Items.GENRE IN ("'+ genre +'")'
            )
            # Add the ARN to the list
            meta_filter_arns.append(createfilter_response['filterArn'])
            print("Creating: " + createfilter_response['filterArn'])

Creating: arn:aws:personalize:us-east-1:136455442858:filter/Film-Noir
Creating: arn:aws:personalize:us-east-1:136455442858:filter/Fantasy
Creating: arn:aws:personalize:us-east-1:136455442858:filter/Western
Creating: arn:aws:personalize:us-east-1:136455442858:filter/Documentary
Creating: arn:aws:personalize:us-east-1:136455442858:filter/Comedy
Creating: arn:aws:personalize:us-east-1:136455442858:filter/Action
Creating: arn:aws:personalize:us-east-1:136455442858:filter/IMAX


También podemos crear 2 filtros de eventos para contenidos vistos y no vistos.

In [14]:
# Create a dataframe for the interactions by reading in the correct source CSV
interactions_df = pd.read_csv(data_dir + '/interactions.csv', sep=',', index_col=0)

# Render some sample data
interactions_df.head(10)

,ITEM_ID,TIMESTAMP,EVENT_TYPE
USER_ID,,,
429,222,828124615,watch
429,227,828124615,click
429,595,828124615,watch
429,592,828124615,watch
429,590,828124615,watch
429,434,828124615,watch
429,421,828124615,watch
429,225,828124615,click
429,343,828124615,click


También podemos crear 2 filtros de eventos para contenidos vistos y no vistos.

In [15]:
createwatchedfilter_response = personalize.create_filter(name='watched',
    datasetGroupArn=dataset_group_arn,
    filterExpression='INCLUDE ItemID WHERE Interactions.event_type IN ("watch")'
    )

createunwatchedfilter_response = personalize.create_filter(name='unwatched',
    datasetGroupArn=dataset_group_arn,
    filterExpression='EXCLUDE ItemID WHERE Interactions.event_type IN ("watch")'
    )


Por último, ya que tenemos el año disponible en nuestros metadatos, crearemos un filtro de década para recomendar sólo películas estrenadas en una década determinada, para este taller elegiremos el cine de los años 70. 

In [16]:
createdecadefilter_response = personalize.create_filter(name='1970s',
    datasetGroupArn=dataset_group_arn,
    filterExpression='INCLUDE ItemID WHERE Items.YEAR >= 1970 AND Items.YEAR < 1980'
    )

Antes de terminar, agregaremos esos filtros a una lista también para que puedan ser utilizados más tarde.

In [17]:
interaction_filter_arns = [createwatchedfilter_response['filterArn'], createunwatchedfilter_response['filterArn']]

In [18]:
decade_filter_arns = [createdecadefilter_response['filterArn']]

In [19]:
%store sims_campaign_arn
%store userpersonalization_campaign_arn
%store rerank_campaign_arn
%store meta_filter_arns
%store interaction_filter_arns
%store decade_filter_arns


Stored 'sims_campaign_arn' (str)
Stored 'userpersonalization_campaign_arn' (str)
Stored 'rerank_campaign_arn' (str)
Stored 'meta_filter_arns' (list)
Stored 'interaction_filter_arns' (list)
Stored 'decade_filter_arns' (list)


Ya está todo listo para pasar al último cuaderno de exploración: `05_Interacting_with_Campaigns_and_Filters.ipynb`. Ábralo desde el navegador y podrá empezar a interactuar con las campañas y obtener recomendaciones.